# 章节3-1 使用Urllib
爬取CSDN（网址：https://edu.csdn.net/course/detail/29493 ）的一个课程页，并自动提取出QQ群。

In [ ]:
import urllib.request
import re

data = urllib.request.urlopen("https://edu.csdn.net/course/detail/29493").read().decode('utf8')
pat = '<span class="realname" data-v-a39d224e>(.*?)</span>'
result = re.compile(pat).findall(str(data))
print(result)

# 章节4-1 timeout设置
循环爬取首页（网址:https://edu.hellobi.com/ ），0.5秒无响应超时异常。

In [ ]:
import urllib.request

# urllib.request.urlretrieve("https://edu.hellobi.com/", filename="1.html")
# urllib.request.urlcleanup()
for i in range(10):
    try:
        file = urllib.request.urlopen("https://edu.hellobi.com/", timeout = 0.5)
        print(file.getcode())
    except Exception as e:
        print("访问{0}出现异常：{1}".format(file.geturl(), str(e)))
# file.geturl()

# 章节4-2 网址有中文
爬取百度（网址：http://www.baidu.com/ ）查找“Python”和“计算机”的查询结果。

In [ ]:
import urllib.request

keywd1 = "python"
keywd2 = urllib.request.quote("计算机")
url1 = "http://www.baidu.com/s?wd=" + keywd1
url2 = "http://www.baidu.com/s?wd=" + keywd2
req1 = urllib.request.Request(url1)
data1 = urllib.request.urlopen(req1).read()
req2= urllib.request.Request(url2)
data2 = urllib.request.urlopen(req2).read()
with open("1.html", "wb") as f:
    f.write(data1)
with open("2.html", "wb") as f:
    f.write(data2)

# 章节4-3 使用post
向网页（网址：http://www.iqianyue.com/mypost/ ）提交。

In [ ]:
import urllib.request
import urllib.parse

url = "http://www.iqianyue.com/mypost/"
mydata = urllib.parse.urlencode({"name":"test@test.com", "pass":"123456jkl"}).encode("utf-8")
req = urllib.request.Request(url, mydata)
print(data, req)
result = urllib.request.urlopen(req).read()
print(result.decode("utf-8"))
# with open("1.html", "wb") as fl:
#     fl.write(result)

# 章节4-4 异常处理

In [ ]:
import urllib.error
import urllib.request

try:
    result = urllib.request.urlopen("http://blog.csdn.net/ss12rwew")
except urllib.error.URLError as e:
    if hasattr(e, "code"):
        print(e.code)
    if hasattr(e, "reason"):
        print(e.reason)

# 章节4-5 使用urllib.request.urlretrieve
爬取新浪新闻（网址：https://news.sina.com.cn/ ）

In [ ]:
import urllib.request
import re

data = urllib.request.urlopen("https://news.sina.com.cn/").read()
data2 = data.decode("utf-8", "ignore")
pat = 'href="(https://news.sina.com.cn/.*?)"'
allurl = re.compile(pat).findall(data2)
# print(allurl)
for i in range(len(allurl)):
    try:
        print("第{}次爬取".format(i+1))
        file = "sinanews/" + str(i+1) + ".html"
        urllib.request.urlretrieve(allurl[i], file)
        print("----成功----")
    except urllib.error.HTTPError as e:
        print(str(e.code) + ": " + e.reason)
    except Exception as e:
        print("失败：{}".format(str(e)))

# 章节5-1 爬虫防屏蔽手段-代理服务器

In [ ]:
import urllib.request
import re

url = "http://blog.csdn.net/"
headers = ("user-agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders = [headers]
urllib.request.install_opener(opener)
data = urllib.request.urlopen(url).read().decode("utf-8", "ignore")
print("output: " + data)

In [ ]:
import urllib.request

def use_proxy(url, proxy_addr):
    proxy = urllib.request.ProxyHandler({"http": proxy_addr})
    opener = urllib.request.build_opener(proxy, urllib.request.HTTPHandler)
    urllib.request.install_opener(opener)
    data = urllib.request.urlopen(url).read().decode("utf-8", "ignore")
    return data

proxy_addr = "47.107.160.99:8118"
url = "http://www.baidu.com"
data = use_proxy(url, proxy_addr)
print("output: " + data)

爬取快代理免费代理IP

In [ ]:
import requests
import time
import random
from lxml import etree
from fake_useragent import UserAgent
from queue import Queue
from threading import Thread


def get_ip():
    while True:
        if not q.empty():
        	# 验证IP是否可用网址
            url = 'http://httpbin.org/get'
            proxies = q.get()
            try:
                html = requests.get(url, headers=headers, proxies=proxies, timeout=5).text
                print('ip可以用')
                with open('ip.txt','a')as f:
                    f.write(str(proxies))
                    f.write('\n')
            except:
                print('ip不可用，下一个\t')
        else:
            break

def main():
    t_list = []
    for i in range(5):
        t = Thread(target=get_ip)
        t_list.append(t)
        t.start()

    for t in t_list:
        t.join()


ip_list = []
q = Queue()
for i in range(1, 10):
    url = 'https://www.kuaidaili.com/free/inha/{}'.format(i)
    print(url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36',
    }
    # proxies = {'http': 'http://211.159.219.225:8118', 'https': 'https://211.159.219.225:8118'}
    # html = requests.get(url, headers=headers,proxies=proxies).text
    html = requests.get(url, headers=headers).text
    # print(html)
    parse_html = etree.HTML(html)
    tr_list = parse_html.xpath('//*[@id="list"]/table/tbody/tr')
    # 延迟访问6到11秒。
    sleep = random.randint(6, 11)
    print(f'等待{sleep}秒')
    time.sleep(sleep)
    print('开始')
    for tr in tr_list[1:]:
        ip = tr.xpath('./td[1]/text()')[0]
        port = tr.xpath('./td[2]/text()')[0]
        proxies = {
            'http': f'http://{ip}:{port}',
            'https': f'https://{ip}:{port}',
        }
        print(proxies)
        # 存入队列
        q.put(proxies)
    main()

print(ip_list)
print("----结束----")

# 章节5-2 图片爬虫
把千图网（https://www.58pic.com/ ）某个频道的所有图片爬下来，高清原版的。(网址变了)

In [ ]:
import urllib.request
import re

headers = ("User-Agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders=[headers]
urllib.request.install_opener(opener)

for i in range(1,2):
#     pageurl = "https://www.58pic.com/haibaomoban/0/id-{}.html".format(str(i*25))
    pageurl = "https://www.58pic.com/tupian/so-0-0-default-0-0-SO-0_10_0_0_0_0_0-0-{}.html".format(str(i))
    data = urllib.request.urlopen(pageurl).read().decode("utf-8", "ignore")
    pat = 'data-original="//(preview.qiantucdn.com/.*?new_nowater)"'
    imglist = re.compile(pat).findall(data)
    for j in range(0, len(imglist)):
        try:
            thisimg = imglist[j]
#             thisimgurl = thisimg + "_1024.jpg"
            thisimgurl = "http://" + thisimg
            file = "img/" + str(i) + str(j) + ".jpg"
            urllib.request.urlretrieve(thisimgurl, filename = file)
            print("----爬取第{}页第{}个图片成功----".format(str(i+1), str(j+1)))
        except urllib.error.URLError as e:
            if hasattr(e, "code"):
                print(e.code)
            if hasattr(e, "reason"):
                print(e.reason)
        except Exception as e:
            print(e)
            

爬取豆瓣电影Top250（https://movie.douban.com/top250 ）。

In [ ]:
import urllib.request
import re

# keyname="短裤"
# key = urllib.request.quote(keyname)
headers = ("User-Agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders=[headers]
urllib.request.install_opener(opener)
print("----开始爬取----")
for i in range(0, 10):
    url = "https://movie.douban.com/top250?start={0}&filter=".format(str(i*25))
    data = urllib.request.urlopen(url).read().decode("utf-8", "ignore")
    pat = 'src="(https://img.*?)" class=""'
    imageurl_list = re.compile(pat).findall(data)
    for j in range(len(imageurl_list)):
        thisimg = imageurl_list[j]
        file = "img/" + str(i) + str(j) + ".jpg"
        urllib.request.urlretrieve(thisimg, filename = file)
    print("----爬取第{}页成功----".format(str(i+1)))

print("爬取网页列表为：" + str(imageurl))

# 章节6-1 抓包分析
爬取腾讯视频（青云志）前20条评论（深度解读）。

In [ ]:
import urllib.request
import re

headers = ("User-Agent", "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36")
opener = urllib.request.build_opener()
opener.addheaders=[headers]
urllib.request.install_opener(opener)
comid, total = "", 0
url = 'https://video.coral.qq.com/filmreviewr/c/upcomment/g7jpq0qd0k8xdca?callback=_filmreviewrcupcommentg7jpq0qd0k8xdca&reqnum=3&source=132&commentid=' + comid + "&_=1605598626157"
while total <= 20:
    try:
        data = urllib.request.urlopen(url, timeout = 10).read().decode()
    except urllib.error.URLError as e:
        if hasattr(e, "code"):
            print(e.code)
        if hasattr(e, "reason"):
            print(e.reason)
    except Exception as e:
        print(e)
    patnext = '"last":"(.*?)"'
    nextid = re.compile(patnext).findall(data)[0]
    pattitle = '"title":"(.*?)",'
    comtitle = re.compile(pattitle).findall(data)
    patcom = '"content":"(.*?)",'
    comdata = re.compile(patcom).findall(data)
    for j in range(len(comdata)):
        total += 1
        if total > 20:
            break
        print("----第{0}条评论----".format(str(total)))
        title = eval('u"' + comtitle[j] + '"') 
        print("标题：" + title) 
        content = eval('u"' + comdata[j] + '"')
        print("内容：" + content)
    url = 'https://video.coral.qq.com/filmreviewr/c/upcomment/g7jpq0qd0k8xdca?callback=_filmreviewrcupcommentg7jpq0qd0k8xdca&reqnum=3&source=132&commentid=' + nextid + "&_=1605598626157"
print("----结束爬虫----")